# C. elegans life cycle model (including dauers)

In [2]:
!pip install mesa 'prompt-toolkit==1.0.15' # For google CoLab


## Defining the model

In [8]:
# Additional functions for model
def p_enter_dauers(T, density):
    '''Probability of entering dauers'''
    if density > 5.0:
        return 1.0
    else:
        return 0.2

def p_exit_dauers(T, density):
    '''Probability of exiting dauers per hour'''
    return 0.001*(1-p_enter_dauers(T, density))

def p_die_in_dauers(T, density):
    '''Probability of dying in dauers per hour'''
    return 0.00001

def sample_p(p):
    if np.random.rand() <= p:
        return True
    else:
        return False
    
def growth_rate(stage, T):
    '''1/(hours until next stage)'''
    return 1./10.

def n_eggs(T):
    '''number of offspring as a function of temperature. Assuming this is set at birth.'''
    return 10.

def max_rate(T):
    '''maximum eggs/hour as a function of temperature.'''
    return 5.0

def egg_rate(stage, T):
    '''Birth rate per hour at a given stage as an integer'''
    return 1

def temp_schedule(hour):
    '''termperature in celcius as a function of the day'''
    return 20.

In [2]:
# Additional functions for collecting data 
# ...

In [3]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import random

# Model
class cElegansWorld(Model):
    """A model with some number of agents."""
    def __init__(self, N, width, height, temp_function):
        self.num_agents = N
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.hour = 0
        self.temp_function = temp_function
        
        # Create agents
        for i in range(self.num_agents):
            a = cElegans(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
        
        self.datacollector = DataCollector(
            #model_reporters={"Hour": "hour"},           
            agent_reporters={"Stage": "stage", 
                             "Dauers": "dauers",
                             "Check": "check_dauers"}) 
    
    def temp(self):
        return self.temp_function(self.hour)
    
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()
        self.hour += 1

# Agent 
class cElegans(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.age = 0 # in hours
        self.stage = 0.0 
        self.check_dauers = False
        self.dauers = False
        self.radius = 3.0
        self.speed = 5
        self.n_eggs = n_eggs(model.temp())
        
    def lay_egg(self):
        self.model.num_agents += 1
        a = cElegans(self.model.num_agents, self.model)
        self.model.schedule.add(a)
        possible_locations = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        birth_location = random.choice(possible_locations)
        self.model.grid.place_agent(a, birth_location)
        
    def move(self):
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False, radius=self.speed)
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def enter_dauers(self):
        T0 = self.model.temp()
        return sample_p(p_enter_dauers(T0, self.find_local_density()))
        
    def exit_dauers(self):
        T0 = self.model.temp()
        return sample_p(p_exit_dauers(T0, self.find_local_density()))
    
    def die_in_dauers(self):
        T0 = self.model.temp()
        return sample_p(p_die_in_dauers(T0, self.find_local_density()))
        
    def find_local_density(self):
        neighbors = self.model.grid.get_neighbors(self.pos, self.radius, False)
        return(len(neighbors) / (np.pi * self.radius**2))

    def step(self):
        T0 = self.model.temp()
        
        # 1. Move
        if (self.stage >= 1) &  ~self.dauers:
            self.move()
        
        # 2. Enter/exit dauers, reproduce, or die
        if (self.stage >= 3) & (self.stage < 4): # Dauers
            if not self.check_dauers:
                if self.enter_dauers():
                    self.dauers = True
                self.check_dauers = True
            if self.dauers & self.exit_dauers():
                self.dauers = False
                self.stage = 4.0
            if self.dauers & self.die_in_dauers():
                self.model.schedule.remove(self)
        elif (self.stage >= 5) & (self.stage < 8): # Reproductive Stages
            for _ in range(egg_rate(self.stage, T0)):
                if self.n_eggs > 0:
                    self.lay_egg()
                    self.n_eggs +- 1
        elif self.stage >= 8: # Death
            self.model.schedule.remove(self)
        
        # 3. Advance time 
        self.age += 1
        if not self.dauers:
            self.stage += growth_rate(self.stage, T0)        

## Running the Model

In [9]:
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import rc
import numpy as np
%matplotlib inline

rc('animation', html='jshtml')
rc('figure', dpi=120)

In [10]:
model = cElegansWorld(10, 100, 100, temp_schedule)

In [ ]:
N_HRS = 500

stages = ['L0', 'L1', 'L2', 'L3', 'L4', 'Rep', 'Rep-EB', 'Rep-EM', 'Rep-EE']
labels = ['Non-dauers', 'dauers']

# First set up the figure, the axes, and the plot element
fig = plt.figure(figsize=(10,4))
grid = plt.GridSpec(2, 2, wspace=0.4, hspace=0.3)
ax1 = fig.add_subplot(grid[0, 1])
ax2 = fig.add_subplot(grid[:, 0])
ax3 = fig.add_subplot(grid[1, 1])
plt.close()

# animation function: this is called sequentially
def animate(i):
    # Step model
    model.step()
    
    # Collect Data
    ag = model.datacollector.get_agent_vars_dataframe()
    
    # Set title
    fig.suptitle("{0} hr".format(model.hour))
    
    # Figure 1
    ax1.cla()
    ax1.hist(ag.loc[i]["Stage"], color='.9', bins=range(10))
    ax1.set_xticks(range(len(stages)))
    ax1.set_xticklabels(stages)
    ax1.set_xlabel('Stages')
     
    # Figure 2
    RANGE = 5
    ax2.cla()
    agent_counts = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        agent_count = len(cell_content)
        agent_counts[x][y] = agent_count
    image = ax2.imshow(agent_counts, interpolation='nearest')
    image.set_clim([0, RANGE])
#     cbar = plt.colorbar(image)
#     cbar.set_clim(vmin=0, vmax=RANGE)
#     cbar_ticks = np.linspace(0., RANGE, num=RANGE+1, endpoint=True)
#     cbar.set_ticks(cbar_ticks) 
#     cbar.draw_all() 
    
    # Figure 3
    ax3.cla()
    N = len(ag.loc[i]["Dauers"])
    N_D = np.sum(ag.loc[i]["Dauers"])
    N_ND = N - N_D
    ax3.bar([0, 1], [N_ND, N_D], color='.9')
    ax3.set_xticks(range(len(labels)))
    ax3.set_xticklabels(labels)

anim = animation.FuncAnimation(fig, animate, frames=N_HRS, interval=200)
anim